In [162]:
import pandas as pd
import numpy as np

In [163]:
a=pd.read_csv('대분류세분류테이블.csv',encoding='cp949')

In [220]:
b=pd.read_csv('17기존고객.csv',encoding='cp949')

In [221]:
c=pd.read_csv('16기존고객.csv',encoding='cp949')

In [165]:
h=a[(a['소비재분류']=='선매품')]

In [166]:
# 선매품 구매횟수
df_pivot_f1 = pd.pivot_table(h,                # 피벗할 데이터프레임
                     index = '고객번호',    # 행 위치에 들어갈 열
                     columns = 'QUARTER',    # 열 위치에 들어갈 열
                     values = '영수증번호',     # 데이터로 사용할 열
                     aggfunc = 'count')   # 데이터 집계함수

In [167]:
df_pivot_f1 = df_pivot_f1.fillna(0)

In [168]:
# 8기 드랍 테이블
df_pivot_f2 = df_pivot_f1.drop(['8Q'],axis=1)

In [169]:
df_pivot_f2['선매품구매횟수'] = df_pivot_f2['1Q'] + df_pivot_f2['2Q'] + df_pivot_f2['3Q'] + df_pivot_f2['4Q'] + df_pivot_f2['5Q']+ df_pivot_f2['6Q'] + df_pivot_f2['7Q']

In [170]:
df_pivot_f3 = df_pivot_f2.drop(['1Q','2Q','3Q','4Q','5Q','6Q','7Q'],axis=1)

In [171]:
# 7기까지 기존고객과 7기까지 총구매횟수 합침
tree=pd.merge(b, df_pivot_f3,on='고객번호', how='left')

In [172]:
tree = tree.fillna(0)

In [173]:
# 17기 구매횟수
tree

,고객번호,선매품구매횟수
0,1,306.0
1,2,262.0
2,3,2.0
3,4,126.0
4,6,158.0
...,...,...
18873,19333,8.0
18874,19334,43.0
18875,19335,127.0
18876,19336,167.0


In [194]:
df_pivot_f3 = df_pivot_f1.drop(['8Q','7Q'],axis=1)

In [195]:
df_pivot_f3['선매품구매횟수'] = df_pivot_f3['1Q'] + df_pivot_f3['2Q'] + df_pivot_f3['3Q'] + df_pivot_f3['4Q'] + df_pivot_f3['5Q']+ df_pivot_f3['6Q']

In [196]:
df_pivot_f3 = df_pivot_f3.drop(['1Q','2Q','3Q','4Q','5Q','6Q'],axis=1)

In [197]:
tree10=pd.merge(c, df_pivot_f3,on='고객번호', how='left')

In [198]:
tree10 = tree10.fillna(0)

In [199]:
# 16기 구매횟수
tree10

,고객번호,선매품구매횟수
0,1,268.0
1,2,219.0
2,3,2.0
3,4,108.0
4,5,48.0
...,...,...
18892,19333,8.0
18893,19334,39.0
18894,19335,116.0
18895,19336,145.0


In [174]:
# 분기별 구매금액
df_pivot_s2 = pd.pivot_table(h,                # 피벗할 데이터프레임
                     index = '고객번호',    # 행 위치에 들어갈 열
                     columns = 'QUARTER',    # 열 위치에 들어갈 열
                     values = '구매금액',     # 데이터로 사용할 열
                     aggfunc = 'sum')   # 데이터 집계함수

In [175]:
df_pivot_s2 = df_pivot_s2.fillna(0)

In [176]:
df_pivot_s3 = df_pivot_s2.drop(['8Q'],axis=1)

In [177]:
# 선매품 등급화
aa=df_pivot_s3['1Q'].sort_values(ascending=False)
cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
cc=pd.concat([aa,cuts],axis=1)

In [178]:
lst=['2Q','3Q','4Q','5Q','6Q','7Q']
for i in lst:
    aa=df_pivot_s3[f'{i}'].sort_values(ascending=False)
    
    cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
    table=pd.concat([aa,cuts],axis=1)  
    cc=pd.merge(cc,table, on='고객번호',how='left')

In [179]:
cc.columns=['선매품1','선매품등급1','선매품2','선매품등급2','선매품3','선매품등급3','선매품4','선매품등급4','선매품5','선매품등급5','선매품6','선매품등급6','선매품7','선매품등급7']

In [180]:
cc = cc.drop(['선매품1','선매품2','선매품3','선매품4','선매품5','선매품6','선매품7'],axis=1)

In [181]:
cc = cc.reset_index()

In [182]:
cc = cc.sort_values('고객번호')

In [183]:
# 7기까지 기존고객과 합침
tree2=pd.merge(b, cc,on='고객번호', how='left')
tree2

,고객번호,선매품등급1,선매품등급2,선매품등급3,선매품등급4,선매품등급5,선매품등급6,선매품등급7
0,1,1,1,1,1,1,1,1
1,2,1,1,1,2,1,2,1
2,3,10,8,9,10,10,10,10
3,4,5,3,3,4,5,5,4
4,6,5,4,4,4,3,3,4
...,...,...,...,...,...,...,...,...
18873,19333,8,10,10,9,10,9,10
18874,19334,9,9,9,8,6,9,6
18875,19335,7,5,6,7,6,6,7
18876,19336,6,3,3,2,3,3,2


In [184]:
# 증감을 구하기 위해 엑셀로 옮김
import csv
tree2.to_excel('선매품17.xlsx',encoding='euc-kr',index=False)

In [185]:
# 구한 증감을 다시 불러옴
z= pd.read_excel('선매품17.xlsx')

In [187]:
z = z.drop(['선매품등급1','선매품등급2','선매품등급3','선매품등급4','선매품등급5','선매품등급6','선매품등급7'],axis=1)

In [188]:
tree3=pd.merge(tree, z,on='고객번호', how='left')

In [189]:
import csv
tree3.to_csv('선매품테이블17기.csv',mode="w",encoding='euc-kr',index=False)

In [200]:
df_pivot_s4 = df_pivot_s2.drop(['8Q','7Q'],axis=1)

In [201]:
# 선매품 등급화
aa=df_pivot_s4['1Q'].sort_values(ascending=False)
cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
cc=pd.concat([aa,cuts],axis=1)

In [202]:
lst=['2Q','3Q','4Q','5Q','6Q']
for i in lst:
    aa=df_pivot_s4[f'{i}'].sort_values(ascending=False)
    
    cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
    table=pd.concat([aa,cuts],axis=1)  
    cc=pd.merge(cc,table, on='고객번호',how='left')

In [203]:
cc.columns=['선매품1','선매품등급1','선매품2','선매품등급2','선매품3','선매품등급3','선매품4','선매품등급4','선매품5','선매품등급5','선매품6','선매품등급6']

In [204]:
cc = cc.drop(['선매품1','선매품2','선매품3','선매품4','선매품5','선매품6'],axis=1)

In [205]:
tree11=pd.merge(c, cc,on='고객번호', how='left')
tree11

,고객번호,선매품등급1,선매품등급2,선매품등급3,선매품등급4,선매품등급5,선매품등급6
0,1,1,1,1,1,1,1
1,2,1,1,1,2,1,2
2,3,10,8,9,10,10,10
3,4,5,3,3,4,5,5
4,5,4,5,4,9,4,6
...,...,...,...,...,...,...,...
18892,19333,8,10,10,9,10,9
18893,19334,9,9,9,8,6,9
18894,19335,7,5,6,7,6,6
18895,19336,6,3,3,2,3,3


In [206]:
import csv
tree11.to_excel('선매품16.xlsx',encoding='euc-kr',index=False)

In [207]:
# 구한 증감을 다시 불러옴
z= pd.read_excel('선매품16.xlsx')

In [208]:
z = z.drop(['선매품등급1','선매품등급2','선매품등급3','선매품등급4','선매품등급5','선매품등급6'],axis=1)

In [210]:
# 7기까지 기존고객과 합침
tree5=pd.merge(tree10, z,on='고객번호', how='left')

In [211]:
import csv
tree5.to_csv('선매품테이블16기.csv',mode="w",encoding='euc-kr',index=False)